In [63]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
# read in data
TNT_postals = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
TNT_postals

[            0                 1  \
 0    Postcode           Borough   
 1         M1A      Not assigned   
 2         M2A      Not assigned   
 3         M3A        North York   
 4         M4A        North York   
 5         M5A  Downtown Toronto   
 6         M5A  Downtown Toronto   
 7         M6A        North York   
 8         M6A        North York   
 9         M7A      Queen's Park   
 10        M8A      Not assigned   
 11        M9A         Etobicoke   
 12        M1B       Scarborough   
 13        M1B       Scarborough   
 14        M2B      Not assigned   
 15        M3B        North York   
 16        M4B         East York   
 17        M4B         East York   
 18        M5B  Downtown Toronto   
 19        M5B  Downtown Toronto   
 20        M6B        North York   
 21        M7B      Not assigned   
 22        M8B      Not assigned   
 23        M9B         Etobicoke   
 24        M9B         Etobicoke   
 25        M9B         Etobicoke   
 26        M9B         Etobi

In [3]:
# use data from sectioned columns with "0" - included all three columns automatically
TNT = TNT_postals[0]
# label columns
TNT.columns = ['Postcode', 'Borough', 'Neighborhood']
# dropped the first row because it contained the column headers
TNT = TNT.iloc[1:]
TNT.head()

,Postcode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [4]:
# exclude all rows where Borough value was Not assigned
TNT = TNT[TNT.Borough != 'Not assigned']
# replace all Not assigned values in Neighborhood with the corresponding Borough value
TNT.Neighborhood.replace('Not assigned',TNT.Borough,inplace=True)
TNT.head(10)

,Postcode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


In [5]:
# grouped rows by common Postcode value, created lists in Neighborhood column
TNT = TNT.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
TNT.head(10)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [6]:
# shape of 103 rows and 3 columns
TNT.shape

(103, 3)

## End of submission 1
--------------------------------------------------------------------------
## Start submission 2

In [19]:
coordinates = pd.read_csv('http://cocl.us/Geospatial_data')
coordinates.columns = ['Postcode', 'Latitude', 'Longitude']
coordinates.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
TNT_coords = pd.merge(TNT, coordinates, how='left', on=['Postcode'])
TNT_coords.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## End Submission 2
-------------------------------------------------------
## Start Submission 3

In [31]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

# dropping str columns to use latitude and longitude float only
TNT_coords_clustering = TNT_coords.drop('Neighborhood', 1)
TNT_coords_clustering = TNT_coords_clustering.drop('Borough', 1)
TNT_coords_clustering = TNT_coords_clustering.drop('Postcode', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(TNT_coords_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 2, 0, 2], dtype=int32)

In [36]:
!conda install -c conda-forge folium=0.5.0
import folium

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [40]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

In [45]:
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi in zip(TNT_coords['Latitude'], TNT_coords['Longitude'], TNT_coords['Neighborhood']):
    label = folium.Popup(str(poi) + ' Cluster ', parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[kclusters-1],
        fill=True,
        fill_color=rainbow[kclusters-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##### observation: downtown Toronto has many neighborhoods right on top of each other, but as you venture out the neighborhoods become more evenly spaced

In [50]:
TNT_coords['Borough'].unique

<bound method Series.unique of 0           Scarborough
1           Scarborough
2           Scarborough
3           Scarborough
4           Scarborough
5           Scarborough
6           Scarborough
7           Scarborough
8           Scarborough
9           Scarborough
10          Scarborough
11          Scarborough
12          Scarborough
13          Scarborough
14          Scarborough
15          Scarborough
16          Scarborough
17           North York
18           North York
19           North York
20           North York
21           North York
22           North York
23           North York
24           North York
25           North York
26           North York
27           North York
28           North York
29           North York
             ...       
73                 York
74                 York
75     Downtown Toronto
76         West Toronto
77         West Toronto
78         West Toronto
79           North York
80                 York
81                 York
82       

In [57]:
northyork_data = TNT_coords[TNT_coords['Borough'] == 'North York'].reset_index(drop=True)
northyork_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493


In [58]:
# create map of North York using latitude and longitude values
map_northyork = folium.Map(location=[43.7797, -79.4156], zoom_start=11)

# add markers to map
for lat, lng, label in zip(northyork_data['Latitude'], northyork_data['Longitude'], northyork_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_northyork)  
    
map_northyork

In [55]:
CLIENT_ID = '4BRJSOWJ3QQ5AX3XMYZO5KXRF4MAYRR35VNRIG1O320NWBU3' # your Foursquare ID
CLIENT_SECRET = '2DSXIDEZVXIBG40O2VACXTBOTPWPJHUUEDNPLPR3W2VTPGWF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4BRJSOWJ3QQ5AX3XMYZO5KXRF4MAYRR35VNRIG1O320NWBU3
CLIENT_SECRET:2DSXIDEZVXIBG40O2VACXTBOTPWPJHUUEDNPLPR3W2VTPGWF


In [59]:
northyork_data.loc[0, 'Neighborhood']

'Hillcrest Village'

In [60]:
neighborhood_latitude = northyork_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = northyork_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = northyork_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Hillcrest Village are 43.8037622, -79.3634517.


In [61]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=4BRJSOWJ3QQ5AX3XMYZO5KXRF4MAYRR35VNRIG1O320NWBU3&client_secret=2DSXIDEZVXIBG40O2VACXTBOTPWPJHUUEDNPLPR3W2VTPGWF&v=20180605&ll=43.8037622,-79.3634517&radius=500&limit=100'

In [64]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c624e5e351e3d1951213984'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4ad9dce6f964a520651b21e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/golfcourse_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1e6941735',
         'name': 'Golf Course',
         'pluralName': 'Golf Courses',
         'primary': True,
         'shortName': 'Golf Course'}],
       'id': '4ad9dce6f964a520651b21e3',
       'location': {'address': '10000 Dufferin Rd',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 197,
        'formattedAddress': ['10000 Dufferin Rd', 'Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.805454826002794,
    

In [65]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [66]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Eagle's Nest Golf Club,Golf Course,43.805455,-79.364186
1,AY Jackson Pool,Pool,43.804515,-79.366138
2,Villa Madina,Mediterranean Restaurant,43.801685,-79.363938
3,Duncan Creek Park,Dog Run,43.805539,-79.360695
4,A.Y. Jackson Secondary School Track,Athletics & Sports,43.805068,-79.366677


In [67]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [68]:
northyork_venues = getNearbyVenues(names=northyork_data['Neighborhood'],
                                   latitudes=northyork_data['Latitude'],
                                   longitudes=northyork_data['Longitude']
                                  )


Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Bedford Park, Lawrence Manor East
Lawrence Heights, Lawrence Manor
Glencairn
Maple Leaf Park, North Park, Upwood Park
Humber Summit
Emery, Humberlea


In [69]:
print(northyork_venues.shape)
northyork_venues.head()

(227, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
3,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
4,Hillcrest Village,43.803762,-79.363452,A.Y. Jackson Secondary School Track,43.805068,-79.366677,Athletics & Sports


In [70]:
northyork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Downsview North, Wilson Heights",17,17,17,17,17,17
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
"CFB Toronto, Downsview East",3,3,3,3,3,3
Don Mills North,5,5,5,5,5,5
Downsview Central,3,3,3,3,3,3
Downsview Northwest,5,5,5,5,5,5
Downsview West,3,3,3,3,3,3
"Emery, Humberlea",2,2,2,2,2,2


In [71]:
# one hot encoding
northyork_onehot = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
northyork_onehot['Neighborhood'] = northyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [northyork_onehot.columns[-1]] + list(northyork_onehot.columns[:-1])
northyork_onehot = northyork_onehot[fixed_columns]

northyork_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Tailor Shop,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hillcrest Village,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
northyork_onehot.shape

(227, 102)

In [74]:
northyork_grouped = northyork_onehot.groupby('Neighborhood').mean().reset_index()
northyork_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Tailor Shop,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,"Bathurst Manor, Downsview North, Wilson Heights",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.058824,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000
1,Bayview Village,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.250000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.040000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,...,0.000000,0.000000,0.04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CFB Toronto, Downsview East",0.0,0.333333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Don Mills North,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Downsview Central,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Downsview Northwest,0.0,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Downsview West,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.333333,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Emery, Humberlea",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Fairview, Henry Farm, Oriole",0.0,0.000000,0.016667,0.000000,0.033333,0.0,0.033333,0.016667,0.00,...,0.016667,0.016667,0.00,0.016667,0.016667,0.016667,0.000000,0.000000,0.016667,0.033333


In [75]:
num_top_venues = 5

for hood in northyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = northyork_grouped[northyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Downsview North, Wilson Heights----
              venue  freq
0       Coffee Shop  0.12
1  Sushi Restaurant  0.06
2          Pharmacy  0.06
3       Pizza Place  0.06
4             Diner  0.06


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                  venue  freq
0           Coffee Shop  0.08
1    Italian Restaurant  0.08
2           Pizza Place  0.08
3  Fast Food Restaurant  0.08
4     Indian Restaurant  0.04


----CFB Toronto, Downsview East----
      venue  freq
0   Airport  0.33
1      Park  0.33
2  Bus Stop  0.33
3     Plaza  0.00
4  Pharmacy  0.00


----Don Mills North----
                  venue  freq
0  Gym / Fitness Center   0.2
1  Caribbean Restaurant   0.2
2                  Café   0.2
3   Japanese Restaurant   0.2
4      Basketball Court   0.2


----Downsv

In [76]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [77]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = northyork_grouped['Neighborhood']

for ind in np.arange(northyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Fried Chicken Joint,Shopping Mall,Frozen Yogurt Shop,Pharmacy,Pizza Place,Deli / Bodega,Bridal Shop,Restaurant,Sandwich Place
1,Bayview Village,Japanese Restaurant,Bank,Café,Chinese Restaurant,Frozen Yogurt Shop,Discount Store,Coffee Shop,Comfort Food Restaurant,General Entertainment,Construction & Landscaping
2,"Bedford Park, Lawrence Manor East",Coffee Shop,Fast Food Restaurant,Pizza Place,Italian Restaurant,Sandwich Place,Comfort Food Restaurant,Café,Butcher,Liquor Store,Juice Bar
3,"CFB Toronto, Downsview East",Park,Airport,Bus Stop,Dog Run,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store
4,Don Mills North,Caribbean Restaurant,Gym / Fitness Center,Café,Basketball Court,Japanese Restaurant,Fried Chicken Joint,Discount Store,Comfort Food Restaurant,Furniture / Home Store,Construction & Landscaping


In [91]:
# set number of clusters
kclusters = 5

northyork_grouped_clustering = northyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(northyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 1, 0, 3, 2, 1, 4, 2, 1], dtype=int32)

In [92]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ValueError: cannot insert Cluster Labels, already exists

In [93]:
northyork_merged = northyork_data

# merge northyork_grouped with northyork_data to add latitude/longitude for each neighborhood
# then drop NaN Cluster Label values which will not process in the next step
northyork_merged = northyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
northyork_merged = northyork_merged[pd.notnull(northyork_merged['Cluster Labels'])]

northyork_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,1.0,Golf Course,Dog Run,Pool,Athletics & Sports,Mediterranean Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,1.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Asian Restaurant,Bakery,Women's Store,Food Court,Gift Shop,Cosmetics Shop
2,M2K,North York,Bayview Village,43.786947,-79.385975,3.0,Japanese Restaurant,Bank,Café,Chinese Restaurant,Frozen Yogurt Shop,Discount Store,Coffee Shop,Comfort Food Restaurant,General Entertainment,Construction & Landscaping
5,M2N,North York,Willowdale South,43.770120,-79.408493,1.0,Restaurant,Ramen Restaurant,Café,Pizza Place,Coffee Shop,Japanese Restaurant,Sandwich Place,Shopping Mall,Pharmacy,Plaza
6,M2P,North York,York Mills West,43.752758,-79.400049,0.0,Park,Bank,Women's Store,Dog Run,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store


In [100]:
# create map
map_clusters = folium.Map(location=[43.7797, -79.4156], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(northyork_merged['Latitude'], northyork_merged['Longitude'], northyork_merged['Neighborhood'], northyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [101]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 0, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,North York,0.0,Park,Bank,Women's Store,Dog Run,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store
8,North York,0.0,Park,Food & Drink Shop,Fast Food Restaurant,Women's Store,Discount Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
13,North York,0.0,Park,Airport,Bus Stop,Dog Run,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store


In [102]:
# cluster with the highest variety of venues
northyork_merged.loc[northyork_merged['Cluster Labels'] == 1, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1.0,Golf Course,Dog Run,Pool,Athletics & Sports,Mediterranean Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
1,North York,1.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Asian Restaurant,Bakery,Women's Store,Food Court,Gift Shop,Cosmetics Shop
5,North York,1.0,Restaurant,Ramen Restaurant,Café,Pizza Place,Coffee Shop,Japanese Restaurant,Sandwich Place,Shopping Mall,Pharmacy,Plaza
7,North York,1.0,Pharmacy,Pizza Place,Grocery Store,Coffee Shop,Butcher,Discount Store,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop
10,North York,1.0,Coffee Shop,Asian Restaurant,Gym,Beer Store,Bike Shop,Clothing Store,Chinese Restaurant,Dim Sum Restaurant,Bus Line,Japanese Restaurant
11,North York,1.0,Coffee Shop,Fried Chicken Joint,Shopping Mall,Frozen Yogurt Shop,Pharmacy,Pizza Place,Deli / Bodega,Bridal Shop,Restaurant,Sandwich Place
12,North York,1.0,Coffee Shop,Miscellaneous Shop,Massage Studio,Bar,Women's Store,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
16,North York,1.0,Grocery Store,Gym / Fitness Center,Athletics & Sports,Discount Store,Liquor Store,Women's Store,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop
17,North York,1.0,Pizza Place,Coffee Shop,Portuguese Restaurant,Hockey Arena,Women's Store,Discount Store,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop
18,North York,1.0,Coffee Shop,Fast Food Restaurant,Pizza Place,Italian Restaurant,Sandwich Place,Comfort Food Restaurant,Café,Butcher,Liquor Store,Juice Bar


In [103]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 2, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,North York,2.0,Business Service,Food Truck,Baseball Field,Women's Store,Dog Run,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
23,North York,2.0,Furniture / Home Store,Baseball Field,Women's Store,Dog Run,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store


In [104]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 3, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,North York,3.0,Japanese Restaurant,Bank,Café,Chinese Restaurant,Frozen Yogurt Shop,Discount Store,Coffee Shop,Comfort Food Restaurant,General Entertainment,Construction & Landscaping
9,North York,3.0,Caribbean Restaurant,Gym / Fitness Center,Café,Basketball Court,Japanese Restaurant,Fried Chicken Joint,Discount Store,Comfort Food Restaurant,Furniture / Home Store,Construction & Landscaping
21,North York,3.0,Park,Construction & Landscaping,Bakery,Basketball Court,Electronics Store,Coffee Shop,Comfort Food Restaurant,Cosmetics Shop,Deli / Bodega,Department Store


In [105]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 4, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,North York,4.0,Grocery Store,Bank,Women's Store,Electronics Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store


## End of Submission 3
## End of Assignment